In [5]:
from openpyxl import load_workbook
import requests.api
import warnings
from openpyxl import Workbook

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import csv
from random import randint
import json, io

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException

import urllib
import urllib3
import requests
import json, io
from bs4 import BeautifulSoup
urllib3.disable_warnings()
headers = {'User-Agent':'Mozilla/5.0'}


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0')
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'/Users/Name/Downloads/Compressed/chromedrives/chromedriver.exe')


def requestspatch(method, url, **kwargs):
    kwargs['verify'] = False
    return _origcall(method, url, **kwargs)

_origcall = requests.api.request
requests.api.request = requestspatch
warnings.warn('Patched requests: SSL verification disabled!')

list0=[]
wb = load_workbook(filename='MasterList (1).xlsx', read_only=True)
ws = wb['ExtractList']
for row in ws.rows:
    list0.append((row[0].value,row[1].value,row[2].value,row[3].value,row[4].value,row[5].value,row[6].value))
del list0[0]

fulldata=[]
for i in range(1,len(list0)):    
    driver.get('https://nycprop.nyc.gov/nycproperty/nynav/jsp/selectbbl.jsp')
    block=driver.find_element_by_name('FBLOCK')
    block.send_keys(int(list0[i][2]))
    lot=driver.find_element_by_name('FLOT')
    lot.send_keys(int(list0[i][3]))
    driver.find_element_by_xpath('/html/body/div/center/table[2]/tbody/tr/td[1]/table/tbody/tr[2]/td[2]/div/table[3]/tbody/tr/td/div/p/table/tbody/tr/td/table/tbody/tr[2]/td/form/table/tbody/tr[7]/td/input[1]').click()
    sleep(5)

    so0=BeautifulSoup(driver.page_source, 'lxml')
    details0=so0.find_all('td',colspan="2")[1]
    details1=details0.find_all('td')
    parcelid=details1[1].text
    data0=[]
    data0.append(parcelid)
    for w in range(0,len(details1)):
        if details1[w].text=='\nProperty Address: ':

            propertyaddress=details1[w+1].text
            propertyaddress=' '.join(propertyaddress.split())
            data0.append(propertyaddress)

            if len(details1)==6:
                owner1=details1[w-1].text
                owner1=owner1.replace('  ',' ')
                print owner1
                data0.append(owner1)

            if len(details1)==8:
                owner1=details1[w-3].text
                owner1=owner1.replace('  ',' ')
                print owner1
                data0.append(owner1)

                owner2=details1[w-1].text
                owner2=owner2.replace('  ',' ')
                data0.append(owner2)
                print owner2
            if len(details1)==10:
                owner1=details1[w-5].text
                owner1=owner1.replace('  ',' ')
                print owner1
                data0.append(owner1)

                owner2=details1[w-3].text
                owner2=owner2.replace('  ',' ')
                data0.append(owner2)
                print owner2
                owner3=details1[w-1].text
                owner3=owner3.replace('  ',' ')
                data0.append(owner3)
                print owner3



    data0
    fulldata.append(data0)

    ass=driver.find_elements_by_tag_name('a')
    for e in range(0,len(ass)):
        qtext=ass[e].text
        if 'Quarterly Property Tax Bill' in qtext:
            print qtext
            print e
            break
    ass[e].click()

    so1=BeautifulSoup(driver.page_source, 'lxml')
    ass=so1.find_all('a')
    len(ass)
    for e in range(0,len(ass)):
        qtext=ass[e].text
        if 'Quarterly Property Tax Bill' in qtext:
            break
    linktodown=ass[e].get('href')

    qtext=(qtext.replace('\n','')).replace('\t','')
    for y in range(0,len(qtext)):
        if qtext[0:y]=='Click here to view your ':
            break
    qtext0=qtext[y:]

    for u in range(0,len(qtext0)):
        if qtext0[u:len(qtext0)-1]==' - Quarterly Property Tax Bill':
            break
    qtext1=qtext0[:u]
    name0=(qtext1.replace(',','')).replace(' ','_')
    name0
    name=name0+parcelid.replace('-','_')
    pdf_name=name+'.pdf'
    pdf_name

    for t in range(0,len(linktodown)):
        if linktodown[t:t+15]=='StatementSearch':
            break
    linktodown=linktodown[t:len(linktodown)]
    linktodown
    intiallinkd='https://nycprop.nyc.gov/nycproperty/'
    download=intiallinkd+linktodown


    response = requests.get(download)
    with open(pdf_name, 'wb') as f:
        f.write(response.content)

wb = Workbook(write_only=True)
ws = wb.create_sheet()

# now we'll fill it with 100 rows x 200 columns
for irow in fulldata:
    ws.append(irow)
# save the file
wb.save('new_big_file.xlsx')